In [7]:
import pandas as pd
import numpy as np

In [8]:
ip_data = pd.read_csv(r"C:\Users\nandi\Documents\ML_CO2_ER\data\Data_computed_properties\filtered_ip.csv")
ip_data.dropna(inplace=True)
ip_data.rename(columns={'solvent ID': 'solvent_ID', 'IP_0-1': 'IP_values'}, inplace=True)

In [9]:
ip_data.head()

,solvent_ID,smiles,IP_values
0,zinc_1345620175,C[C@H](N[C@@H]1CCCC[C@H]1F)[C@@H](C)O,0.724053
1,zinc_674912368,COC[C@H](C)N1C[C@@H]2CCC[C@H]2C1,0.255187
2,zinc_644654185,CC[C@@H](O)CNCC(C)(F)F,1.070564
3,zinc_132906027,CC[C@@H](NCCOCC(C)C)C1CC1,0.763281
4,zinc_156549661,C[C@H](O)CN1CCC[C@H]1C,0.313528


In [10]:
df = pd.read_csv(r"C:\Users\nandi\Documents\ML_CO2_ER\data\df_merged.csv")
print(df.head())
df = ip_data.merge(df, on=['solvent_ID'], how="left")

       solvent_ID                      smiles  Binding_Energy_eV  \
0    zinc_5593217       COCCNC(=O)[C@@H](CC)C          -0.246414   
1  zinc_400210175     CO[C@H](C1CC1)CN1CCCCC1          -0.239933   
2    zinc_5696920                   NOCCN(C)C          -0.219656   
3  zinc_685002938  C[C@@H](CN(CC1CC1)CC(F)F)O          -0.325154   
4   zinc_52602135          CNC[C@@H](C(C)C)OC          -0.288984   

   solubility_CO2 [mol/L]  viscosity [Pa s]  nAcid  nBase    SpAbs_A  \
0                0.097848          0.011764    0.0    0.0  12.906117   
1                0.117575          0.007448    0.0    1.0  17.343700   
2                0.097619          0.002281    0.0    1.0   7.727407   
3                0.088216          0.025603    0.0    1.0  14.943352   
4                0.131763          0.001110    0.0    1.0  10.199109   

    SpMax_A  SpDiam_A  ...   lumo_eV  homo_lumo_gap_eV  polarizability_ang3  \
0  2.122425  4.244850  ... -6.330932        181.930993           748.130500   


In [11]:
df.head()

,solvent_ID,smiles_x,IP_values,smiles_y,Binding_Energy_eV,solubility_CO2 [mol/L],viscosity [Pa s],nAcid,nBase,SpAbs_A,...,lumo_eV,homo_lumo_gap_eV,polarizability_ang3,min_partial_charge,max_partial_charge,min_abs_partial_charge,max_abs_partial_charge,vib_freq_min_cm1,vib_freq_max_cm1,vib_freq_count
0,zinc_1345620175,C[C@H](N[C@@H]1CCCC[C@H]1F)[C@@H](C)O,0.724053,C[C@H]([C@@H](N[C@@H]1CCCC[C@H]1F)C)O,-0.214742,0.079341,0.150835,0.0,1.0,15.711170,...,-4.168789,162.745678,849.696107,-0.819411,0.453256,0.172385,0.819411,51.2361,3727.7322,93.0
1,zinc_674912368,COC[C@H](C)N1C[C@@H]2CCC[C@H]2C1,0.255187,COC[C@@H](N1C[C@H]2[C@H](C1)CCC2)C,-0.214342,0.094735,0.011400,0.0,1.0,16.309342,...,1.917791,150.631754,924.031534,-0.779818,0.232699,0.000737,0.779818,35.7164,3128.8524,96.0
2,zinc_644654185,CC[C@@H](O)CNCC(C)(F)F,1.070564,CC[C@H](CNCC(F)(F)C)O,-0.212998,0.096515,0.022542,0.0,1.0,11.751232,...,-1.747485,177.540066,666.772267,-0.824957,0.633432,0.059057,0.824957,43.3486,3716.1399,72.0
3,zinc_132906027,CC[C@@H](NCCOCC(C)C)C1CC1,0.763281,CC[C@H](C1CC1)NCCOCC(C)C,-0.215950,0.079947,0.004861,0.0,1.0,17.214239,...,2.125120,166.188816,1044.297027,-0.927964,0.380666,0.011509,0.927964,22.3711,3510.5041,111.0
4,zinc_156549661,C[C@H](O)CN1CCC[C@H]1C,0.313528,C[C@@H](CN1CCC[C@H]1C)O,-0.198320,0.088578,0.081929,0.0,1.0,11.731206,...,0.436871,157.318107,709.790668,-0.610850,0.449006,0.051486,0.610850,43.4119,3722.5285,75.0


In [ ]:
# --- Start with original features ---
meta_cols = ['solvent_ID', 'smiles_']
target = 'IP_values'
X = df.drop(columns=meta_cols + [target])
y = df[target]

# Step 1.1: Remove constant or quasi-constant features
constant_thresh = 0.98  # 98% of rows have same value
quasi_constant = [col for col in X.columns if X[col].value_counts(normalize=True).values[0] > constant_thresh]
X_filtered = X.drop(columns=quasi_constant)

print(f"Removed {len(quasi_constant)} quasi-constant features")

# Step 1.2: Remove highly correlated features
corr_matrix = X_filtered.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

high_corr = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
X_filtered = X_filtered.drop(columns=high_corr)

print(f"Removed {len(high_corr)} highly correlated features")

# Output the shape after filtering
print("Remaining features:", X_filtered.shape[1])


Removed 356 quasi-constant features
Removed 569 highly correlated features
Remaining features: 493


In [7]:
X_filtered["solvent_ID"] = df["solvent_ID"].copy()
X_filtered["smiles"] = df["smiles"].copy()
X_filtered["IP_values"] = df["IP_values"].copy()

In [8]:
X_filtered.to_csv(r"C:\Users\nandi\Documents\ML_CO2_ER\data\df_filtered_ip.csv", index=False)

In [9]:
X_filtered.head()

,Binding_Energy_eV,solubility_CO2 [mol/L],viscosity [Pa s],nBase,SpAbs_A,SpMax_A,SpDiam_A,SpMAD_A,VE1_A,VE2_A,...,homo_lumo_gap_eV,min_partial_charge,max_partial_charge,min_abs_partial_charge,max_abs_partial_charge,vib_freq_min_cm1,vib_freq_max_cm1,solvent_ID,smiles,IP_values
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zinc_1345620175,C[C@H](N[C@@H]1CCCC[C@H]1F)[C@@H](C)O,0.724053
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zinc_674912368,COC[C@H](C)N1C[C@@H]2CCC[C@H]2C1,0.255187
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zinc_644654185,CC[C@@H](O)CNCC(C)(F)F,1.070564
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zinc_132906027,CC[C@@H](NCCOCC(C)C)C1CC1,0.763281
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zinc_156549661,C[C@H](O)CN1CCC[C@H]1C,0.313528
